In [1]:
import neptune

neptune.init(
    project_qualified_name='arsde/J22TAO',
)

neptune.create_experiment()

Info (NVML): RM has detected an NVML/RM version mismatch.. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/arsde/J22TAO/e/JTAO-66


Experiment(JTAO-66)

In [2]:
import os

def init(run_name):
    os.system('mkdir models/fcdnn_tun/{}'.format(run_name))
    os.system('mkdir models/fcdnn_tun/{}/models_saved'.format(run_name))
    os.system('mkdir models/fcdnn_tun/{}/results'.format(run_name))
    
run_name = input()
init(run_name)

JTAO-66


In [3]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats
from sklearn.preprocessing import StandardScaler

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

import plotly.io as pio
pio.templates.default = 'plotly_white'

import warnings
warnings.filterwarnings("ignore")

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2022-12-27 16:01:06.935283: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14304089616391529593
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 31595870336
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6437530694806181326
physical_device_desc: "device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:01:01.0, compute capability: 7.0"
]


2022-12-27 16:01:09.090252: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-27 16:01:09.094757: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-27 16:01:09.096892: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-12-27 16:01:09.624050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-27 16:01:09.625188: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01

In [7]:
path = "/mnt/cephfs/ml_data/TAO_detsim_J22/"
data = pd.read_csv(f'{path}processed_data/ProcessedTrain/ProcessedTrain.csv.gz')

In [8]:
# opt_features = list(data.columns[:91])
opt_features = list(np.load('feature_selection/opt_features.npz')['a'])

opt_features += ['edep']

In [9]:
from neptunecontrib.api import log_csv

log_csv('features_set', pd.DataFrame(opt_features[:-1], columns=['feature']))

In [10]:
FC_cut = 0.65

data = data.reset_index(drop=True)
data = data[data['edepR'] < FC_cut][opt_features]
data.shape

(1970550, 26)

In [11]:
random_state = np.random.randint(10000)
print(random_state)

1656


In [12]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=random_state)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, concatenate, Input
from tensorflow.keras.layers import Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping

kernel_initializer_input = ['normal', 'lecun_normal', 'uniform']
kernel_initializer_hidden = ['normal', 'lecun_normal', 'uniform']
activations = ['relu', 'elu', 'selu']
optimizers = ['adam', 'rmsprop', 'SGD']
schedules = ['None', 'ExponentialDecay']
units_input_list = [8, 16, 32, 64, 128, 256, 512]
units_in_hidden_layer_list = [8, 16, 32, 64, 128, 256, 512]
num_hidden_layers_list = [4, 8, 16, 32]

def build_model(hp):
    input_features = Input(shape=X.shape[1])
    
    units_input = hp.Choice('units_input', units_input_list, default=128)
    units_in_hidden_layer = hp.Choice('units_in_hidden_layer', units_in_hidden_layer_list, default=128)
    num_hidden_layers = hp.Choice('num_hidden_layers', num_hidden_layers_list, default=8)
    kernel_initializers_input = hp.Choice('kernel_initializers_input', kernel_initializer_input)  
    kernel_initializers_hidden = hp.Choice('kernel_initializers_hidden', kernel_initializer_hidden)  

    activation = hp.Choice('activation', values=activations)      
    optimizer = hp.Choice('optimizer', values=optimizers)
    
    lr = hp.Float('lr', min_value=1e-4, max_value=1e-2, default=1e-3, sampling='LOG')
    decay_lr = hp.Choice('decay_lr', schedules)
        
    batch_norm = hp.Choice('batch_norm', [False, True])
       
    if decay_lr == 'ExponentialDecay':
        decay_steps = hp.Int('decay_steps', min_value=500, max_value=20000, default=2000)
        decay_rate = hp.Float('decay_rate', min_value=0.1, max_value=0.9, default=0.8)

        lr = ExponentialDecay(
            initial_learning_rate=lr,
            decay_steps=decay_steps,
            decay_rate=decay_rate
        )

    x = Dense(units=units_input,
              kernel_initializer=kernel_initializers_input,
              activation=activation
            )(input_features)
    if batch_norm:
        x = BatchNormalization()(x)

    for i in range(num_hidden_layers):
        x = Dense(units=units_in_hidden_layer,
                  kernel_initializer=kernel_initializers_hidden,
                  activation=activation
            )(x)
        if batch_norm:
            x = BatchNormalization()(x)

    output = Dense(units=1,
                   kernel_initializer='normal',
                   activation='linear')(x)

    model = Model(inputs=input_features,
                  outputs=output,
                  name='Model')

    if optimizer == 'adam':
#         beta_1 = hp.Float('beta_1', min_value=0.5, max_value=0.99, default=0.9)
#         beta_2 = hp.Float('beta_2', min_value=0.5, max_value=0.999, default=0.999)
        optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    elif optimizer == 'rmsprop':
        rho = hp.Float('rho', min_value=1e-1, max_value=1, default=0.9)
        momentum = hp.Float('momentum', min_value=0, max_value=1, default=0.9)
        optimizer = RMSprop(learning_rate=lr, rho=rho, momentum=momentum)
    elif optimizer == 'SGD':
        momentum = hp.Float('momentum', min_value=0, max_value=1, default=0.9)
        nesterov = hp.Boolean('nesterov', default=True)
        optimizer = SGD(learning_rate=lr, momentum=momentum, nesterov=nesterov)

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.MeanAbsolutePercentageError(),
        metrics=[
                tf.keras.metrics.MeanAbsolutePercentageError(name='mape'),
                'mse',
                'mae'
            ])
    
    model.summary()
    return model

In [14]:
import neptunecontrib.monitoring.kerastuner as npt_utils
from keras_tuner.tuners import BayesianOptimization, Hyperband


# class BayesianOptimizationBatchSize(BayesianOptimization):
#     def run_trial(self, trial, *args, **kwargs):
#         kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 128, 2048, step=128)
#         super(BayesianOptimizationBatchSize, self).run_trial(trial, *args, **kwargs)
#         return super(BayesianOptimizationBatchSize, self).run_trial(trial, *args, **kwargs)

tuner = BayesianOptimization(#BayesianOptimizationBatchSize(
    build_model,
    objective='val_mape',
    max_trials=100,
    seed=random_state,
    directory=f'models/fcdnn_tun/{run_name}/saved_networks',
    project_name=run_name,
    logger=npt_utils.NeptuneLogger()
)

2022-12-27 16:04:21.151959: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-27 16:04:21.152320: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-27 16:04:21.153462: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:01.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-12-27 16:04:21.153542: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-12-27 16:04:21.153573: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-12-27 16:04:21.153593: I tensorflow/stream_executor/platform/de

Model: "Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               3328      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512 

In [15]:
monitor = EarlyStopping(
    monitor="val_mape",
    patience=20,
    mode='min',
#     restore_best_weights=True,
    baseline=3
)

In [ ]:
tuner.search(X_train_scaled,
             y_train,
             validation_data=(X_val_scaled, y_val),
             epochs=500, 
             batch_size=512,
             callbacks=[monitor],
             verbose=2
)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
256               |?                 |units_input
256               |?                 |units_in_hidden_layer
32                |?                 |num_hidden_layers
lecun_normal      |?                 |kernel_initializers_input
normal            |?                 |kernel_initializers_hidden
elu               |?                 |activation
SGD               |?                 |optimizer
0.0019687         |?                 |lr
ExponentialDecay  |?                 |decay_lr
1                 |?                 |batch_norm

Model: "Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
dense (Dense)                (None, 256)               6656      
_____________________________________________

2022-12-27 16:04:23.070570: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-27 16:04:23.071367: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2194840000 Hz
2022-12-27 16:04:29.245257: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


3464/3464 - 133s - loss: 6.9682 - mape: 6.9682 - mse: 0.5135 - mae: 0.3262 - val_loss: 3.0365 - val_mape: 3.0365 - val_mse: 0.0233 - val_mae: 0.1177
Epoch 2/500
3464/3464 - 127s - loss: 4.6299 - mape: 4.6299 - mse: 0.0841 - mae: 0.2141 - val_loss: 6.8150 - val_mape: 6.8150 - val_mse: 0.1098 - val_mae: 0.3116
Epoch 3/500
3464/3464 - 126s - loss: 3.9519 - mape: 3.9519 - mse: 0.0637 - mae: 0.1865 - val_loss: 8.0651 - val_mape: 8.0651 - val_mse: 0.0769 - val_mae: 0.2199
Epoch 4/500
3464/3464 - 128s - loss: 3.4715 - mape: 3.4715 - mse: 0.0544 - mae: 0.1699 - val_loss: 2.9092 - val_mape: 2.9092 - val_mse: 0.0338 - val_mae: 0.1392
Epoch 5/500
3464/3464 - 127s - loss: 3.0940 - mape: 3.0940 - mse: 0.0480 - mae: 0.1583 - val_loss: 1.6482 - val_mape: 1.6482 - val_mse: 0.0212 - val_mae: 0.1025
Epoch 6/500
3464/3464 - 128s - loss: 2.7517 - mape: 2.7517 - mse: 0.0435 - mae: 0.1471 - val_loss: 2.0044 - val_mape: 2.0044 - val_mse: 0.0153 - val_mae: 0.0942
Epoch 7/500
3464/3464 - 128s - loss: 2.5533 - 

In [ ]:
tuner.results_summary()
npt_utils.log_tuner_info(tuner)

In [ ]:
model = tuner.get_best_models()[0]
model.save("models/fcdnn_tun/{0}/models_saved/{0}.h5".format(run_name))

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
models = [model]

In [ ]:
energies = [0.1, 0.3, 0.6] + list(range(1, 10))
y_true_array = []
y_pred_array = []
for j in tqdm(range(len(models)), "Options..."):
    y_true = []
    y_pred = []
    for energy in tqdm(energies):
        test = pd.read_csv(f'{path}processed_data/ProcessedTest/{energy}MeV.csv.gz')
        test = test[test['edepR'] < FC_cut][opt_features]
        edep = np.array(test['edep'])
        
        X_test = test.iloc[:, :-1]
        X_test = scaler.transform(X_test)
        edep_preds = models[j].predict(X_test).flatten()
        
        y_true.append(edep)
        y_pred.append(edep_preds)
    y_true_array.append(y_true)
    y_pred_array.append(y_pred)

In [ ]:
from eli5.permutation_importance import get_score_importances 
from sklearn.metrics import mean_squared_error

def score(X, y):
    y_pred = model.predict(X)
    return mean_squared_error(y, y_pred)

base_score, score_decreases = get_score_importances(score, X_val_scaled[:500], y_val[:500])
feature_importances = np.mean(score_decreases, axis=0)

In [ ]:
fi = dict(zip(opt_features, feature_importances))
fi = dict(sorted(fi.items(), key=lambda item: item[1]))
df_fi = pd.DataFrame.from_dict(fi, orient='index', columns=['Permutation Importance'])
df_fi

In [ ]:
from neptunecontrib.api import log_table
log_table('output/permutation_importance', df_fi)

In [ ]:
diffs = np.array([
    [y_pred_array[j][i] - y_true_array[j][i] for i in range(len(y_pred_array[0]))]
    for j in range(len(y_pred_array))
])

In [ ]:
energies = [0.1, 0.3, 0.6] + list(range(1, 10))
energies = np.array([1.022+i for i in energies]).round(5)
energies

In [ ]:
from scipy import stats

a_array = []
errors_array = []
for k in range(diffs.shape[0]):
    a = []
    e = []
    for i in range(diffs.shape[1]):
        fig, ax = plt.subplots()
        nbins = 150
        n, bins, patches = ax.hist(diffs[k][i], nbins, density=True, facecolor = 'grey', alpha = 0.5, label='before');
        plt.close(fig)
        centers = (0.5*(bins[1:]+bins[:-1]))
        pars, cov = curve_fit(lambda x, mu, sig : norm.pdf(x, loc=mu, scale=sig), centers, n, p0=[0,1])  
        a.append(pars)
        e.append(cov)
    a_array.append(a)
    errors_array.append(e)

In [ ]:
from neptunecontrib.api import log_chart
pio.templates.default = 'plotly_white'

for k in range(len(models)):
    fig = go.Figure()

    for i in range(len(diffs[k])): 
        x = np.linspace(diffs[k][i][:22500].min(), diffs[k][i][:22500].max(), 100)
        p = stats.norm.pdf(x, a_array[k][i][0], a_array[k][i][1])

        fig.add_trace(
            go.Scatter(x=x,y=p,
                       mode='lines',
                       name='mu={:.3f} +- {:.3f}, sigma={:.3f} +- {:.3f}'.format(
                         a_array[k][i][0], np.sqrt(errors_array[k][i][0][0]),
                         a_array[k][i][1], np.sqrt(errors_array[k][i][1][1])),
                        visible = (i==0)
                    )
        )

    for i in range(len(diffs[k])): 
        fig.add_trace(
            go.Histogram(
                x=diffs[k][i][:22500],
                xbins=dict(size=0.005),
                showlegend=False,
                marker=dict(
                    color='darkred',
                    line=dict(
                        color='black',
                        width=2,
                    )
                ),
                histnorm='probability density',
                visible = (i==0)
            )
        )

    buttons = []
    for N in range(0, len(diffs[k])): 
        buttons.append(
            dict(
                 args=['visible', [False]*N + [True] + [False]*(len(diffs[k])-1-N)],
                     label='Energy =  {} MeV'.format(energies[N]),
                 method='restyle'
            )
        )

    fig.update_layout(

        xaxis = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
        ),

        yaxis = dict(
            showline=True,
            ticks='outside',
            mirror=True,
            linecolor='black',
            tick0=0,
    #             dtick=1,
            showgrid=True,
            gridcolor='grey',
            gridwidth=0.25,
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=0.25
            ),
    )


    fig.update_layout(
        xaxis_title=r"$$E_{rec} - E_{true}$$",
    #         yaxis_title="y",
        showlegend=True,
        updatemenus=list([
            dict(
                x=0.5,
                y=1.2,
                yanchor='top',
                buttons=buttons
            ),
        ]),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.05,
            xanchor="right",
            x=1
        )
    )

    fig.show()
    log_chart('output/Result_distributions.pdf', fig)

In [ ]:
def plot_results(appr=False):
    
    fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.01,
                        row_width=[0.25, 0.75]
    )
    for k in range(diffs.shape[0]):
        fig.add_trace(
            go.Scatter(
                x=energies,
                y=res[k],
                mode='markers',
                marker=dict(
                    color=colors[k],
                    symbol=symbols[k]
                ),
                showlegend=True,
                error_y=dict(
                    type='data',
                    width=10,
                    array=error_sigma[k],
                    visible=True
                ),
                name=names[k]
            ), row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=energies,
                y=bias[k],
                mode='markers',
                showlegend=False,
                marker=dict(
                    color=colors[k],
                    symbol=symbols[k]
                ),
                error_y=dict(
                        type='data',
                        width=10,
                        array=error_mu[k],
                        visible=True
                ),
                name=names[k]
            ), row=2, col=1
        )

    if appr:
        for k in range(diffs.shape[0]):
            fig.add_trace(
                go.Scatter(
                    x=x_lin,
                    y=func(x_lin, a[k], b[k], c[k]),
                    mode='lines',
                    line=dict(
                    ),
                    opacity=0.5,
                    showlegend=False,
                    name=names[k],
                    marker=dict(
                        color=colors[k]
                    )
                ), row=1, col=1
            )

    xaxis = dict(
        showline=True,
        ticks='outside',
        mirror=True,
        tick0=1,
        dtick=1,
        linecolor='black',
        showgrid=True,
        gridcolor='grey',
        gridwidth=0.25,
    )

    yaxis = lambda range: dict(
        showline=True,
        ticks='outside',
        mirror=True,
        linecolor='black',
        range=range,
        showgrid=True,
        gridcolor='grey',
        gridwidth=0.25,
        zeroline=True,
        zerolinecolor='black',
        zerolinewidth=0.25
    )

    fig.update_layout(
        xaxis2_title="Deposited energy, MeV",
        yaxis1_title="Resolution, %",
        yaxis2_title="Bias, %",

        xaxis1 = xaxis,
        xaxis2 = xaxis,
        yaxis1 = yaxis([0.5, 2.5]),
        yaxis2 = yaxis([-0.3, 0.3]),

        showlegend=True,
        font=dict(
                family="Times New Roman",
                size=18,
        ),
        legend=dict(
            x=0.85,
            y=0.99,
            title_font_family="Times New Roman",
            font=dict(
                family="Times New Roman",
                size=18,
                color="black"
            ),
            bordercolor="Black",
            borderwidth=2
        )
    )

    fig.show()
    if appr:
        pio.write_image(
            fig, 'models/fcdnn_tun/{}/results/appr_results.pdf'.format(run_name),
            width=900, height=600)
        log_chart('{}/results/appr_results.pdf'.format(run_name), fig)
    else:
        pio.write_image(
            fig, 'models/fcdnn_tun/{}/results/results.pdf'.format(run_name),
            width=900, height=600)
        log_chart('{}/results/results.pdf'.format(run_name), fig)

In [ ]:
colors = ['royalblue']
symbols = ['star-square']
names = ['FCDNN']

In [ ]:
error_sigma = []
for k in range(diffs.shape[0]):
    error = [100 * np.sqrt(errors_array[k][i][1][1]) / energies[i] for i in range(len(energies))]
    error_sigma.append(error)
    
error_mu = []
for k in range(diffs.shape[0]):
    error = [100 * np.sqrt(errors_array[k][i][0][0]) / energies[i] for i in range(len(energies))]
    error_mu.append(error)

res = []
bias = []
for k in range(diffs.shape[0]):
    sigma = [100 * a_array[k][i][1] / energies[i] for i in range(len(energies))]
    mu = [100 * a_array[k][i][0] / energies[i] for i in range(len(energies))]
    res.append(sigma)
    bias.append(mu)
    
plot_results()

In [ ]:
def a(x, a):
    return np.sqrt((a/x**0.5)**2)


def b(x, b):
    b_list = []
    b_list.append(np.sqrt(b**2))
    return b_list*len(x)


def c(x, c):
    return np.sqrt((c/x)**2)


def func(x, a, b, c):
    return np.sqrt((a/x**0.5)**2 + b**2 + (c/x)**2) 


def approximated(x, y, yerr):
    popt, pcov = curve_fit(func, x, y, sigma=yerr, maxfev=10**8, bounds=([0, 0, 0], [5, 5, 5]))
    a, b, c = popt
    return func(x, a, b, c), popt, pcov

In [ ]:
from statsmodels.stats.moment_helpers import cov2corr

y_approximated_array = []
coefs_array = []
errors_array = []
corr_matrixes = []
for i in range(diffs.shape[0]):
    y_approximated, coefs, pcov = approximated(
        energies, res[i], error_sigma[i])
    y_approximated_array.append(y_approximated)
    coefs_array.append(coefs)
    errors_array.append(np.sqrt(abs(pcov.diagonal())))
    corr_matrixes.append(cov2corr(pcov))

corr_matrixes = np.array(corr_matrixes)

In [ ]:
reindex = [0, 3, 1, 4, 2, 5]
coefs_df = pd.DataFrame(
    np.hstack((coefs_array, errors_array))
)[reindex]
coefs_df.columns = ['a', r'$\Delta a$', 'b', r'$\Delta b$', 'c', r'$\Delta c$']

a = np.array(coefs_array).T[0]
b = np.array(coefs_array).T[1]
c = np.array(coefs_array).T[2]

In [ ]:
x_lin = np.linspace(0.9, 10.1, 1000)
plot_results(appr=True)

In [ ]:
coefs_df = coefs_df.round(3)
coefs_df.index = names
coefs_df[r'$\tilde{a}$'] = np.sqrt((coefs_df['a']**2 + coefs_df['b']**2 + coefs_df['c']**2))
coefs_df[r'$\Delta \tilde{a}$'] = np.sqrt( (coefs_df['a']*coefs_df[r'$\Delta a$'])**2 + \
                                           (coefs_df['b']*coefs_df[r'$\Delta b$'])**2 + \
                                           (coefs_df['c']*coefs_df[r'$\Delta c$'])**2) / coefs_df[r'$\tilde{a}$']
coefs_df[r'$\Delta \tilde{a}$'] = np.sqrt(
    coefs_df[r'$\Delta \tilde{a}$']**2 + 2 * (
        coefs_df['a'] * coefs_df['b'] / coefs_df[r'$\tilde{a}$']**2 *\
        corr_matrixes[:, 0, 1] * coefs_df[r'$\Delta a$'] * coefs_df[r'$\Delta b$'] +\
        
        coefs_df['a'] * coefs_df['c'] / (coefs_df[r'$\tilde{a}$']**2) *\
        corr_matrixes[:, 0, 2] * coefs_df[r'$\Delta a$'] * coefs_df[r'$\Delta c$'] +\

        coefs_df['b'] * coefs_df['c'] / coefs_df[r'$\tilde{a}$']**2 *\
        corr_matrixes[:, 1, 2] * coefs_df[r'$\Delta b$'] * coefs_df[r'$\Delta c$']
    )
)
coefs_df

In [ ]:
log_table('output/coefs_df', coefs_df)
log_metric('a_tilde', coefs_df[r'$\tilde{a}$']['FCDNN'])
log_metric('a_tilde_std', coefs_df[r'$\Delta \tilde{a}$']['FCDNN'])

In [ ]:
#coefs_df.to_csv(f'models/fcdnn_tun/{run_name}/results/params.csv')